In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))

In [2]:
import ee 
import geemap

ee.Initialize(project="ee-india-reservoirs")

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [3]:
from src.et_green.compute_et_green import compute_et_green, calculate_band_std_dev
from src.et_green.filter_nutzungsflaechen import (
    get_crops_to_exclude,
    get_rainfed_reference_crops,
    create_crop_filters,
    filter_crops,
    add_double_cropping_info,
    get_unique_nutzung,
)
from src.et_green.exporting_utils import process_et_green, prepare_rainfed_fields

from utils.ee_utils import (
    back_to_float,
    back_to_int,
    export_image_to_asset,
    print_value_ranges,
    is_image_empty,
    fill_gaps_with_zeros,
    normalize_string_client,
)

from utils.date_utils import print_collection_dates, merge_same_date_images

---

## Constants

In [48]:
YEAR = 2018

PATH_TO_AOI = "projects/thurgau-irrigation/assets/FribourgAndVaud/broye_bounding_box"

PATH_TO_ET_PRODUCT = f"projects/thurgau-irrigation/assets/FribourgAndVaud/ET_products/WaPOR_10m_{YEAR}"
# PATH_TO_ET_PRODUCT = f"projects/thurgau-irrigation/assets/FribourgAndVaud/ET_products/decadal_Landsat_10m"


PATH_TO_DOUBLE_CROPPING_COLLECTION = f"projects/thurgau-irrigation/assets/FribourgAndVaud/DoubleCropping/crop_vegetation_period_broye_{YEAR}"
PATH_TO_JURISDICTION = (
    f"projects/thurgau-irrigation/assets/FribourgAndVaud/elevation_bands_broye_v2_WGS84"
)
PATH_TO_LANDUSE = (
    f"projects/thurgau-irrigation/assets/FribourgAndVaud/filtered_permanent_pastures"
)
LANDUSE_PROPERTY_NAME = "nutzung"

ET_BAND_NAME ="downscaled"#"ET"
# ET_BAND_NAME ="ET"
SCALING_FACTOR = 100  # scaling factor for the ET band to convert it back to float
DYNAMIC = False #True #False, for wapor
SCALING_FACTOR_PROPERTY_NAME = "days" if DYNAMIC else None
ET_BAND_RESOLUTION = 10  # 30# in meters
NUMBER_OF_IMAGES = 21
TEMPORAL_RESOLUTION = "dekadal"

MINIMUM_FIELD_SIZE = 10000  # in square meters

# ET_GREEN_ASSET_PATH = f"projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_{TEMPORAL_RESOLUTION}_from_Landsat_30m_v2"
ET_GREEN_ASSET_PATH = f"projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_{TEMPORAL_RESOLUTION}_from_WaPOR_10m_v2"
# ET_GREEN_STD_ASSET_PATH = f"projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_std_{TEMPORAL_RESOLUTION}_from_WaPOR_10m_v2"

# print(ET_GREEN_STD_ASSET_PATH)

## 1. Load Assets

In [5]:
aoi = ee.FeatureCollection(PATH_TO_AOI).geometry().simplify(500).buffer(100)

In [49]:
double_cropping_image = ee.Image(PATH_TO_DOUBLE_CROPPING_COLLECTION)

In [50]:
et_collection = (
    ee.ImageCollection(PATH_TO_ET_PRODUCT)
    .filterDate(f"{YEAR}-01-01", f"{YEAR}-12-31")
    .map(
        lambda img: back_to_float(
            img, SCALING_FACTOR, DYNAMIC, SCALING_FACTOR_PROPERTY_NAME
        )
    )
).sort("system:time_start")

et_collection_list = et_collection.toList(et_collection.size())


# Sanity checks:
print_collection_dates(et_collection)
print(f"Sizing of the ET collection: {et_collection.size().getInfo()}")

Dates of images in the collection:
2018-04-01
2018-04-11
2018-04-21
2018-05-01
2018-05-11
2018-05-21
2018-06-01
2018-06-11
2018-06-21
2018-07-01
2018-07-11
2018-07-21
2018-08-01
2018-08-11
2018-08-21
2018-09-01
2018-09-11
2018-09-21
2018-10-01
2018-10-11
2018-10-21
Sizing of the ET collection: 21


In [9]:
# image = ee.Image(et_collection_list.get(12))

# Map = geemap.Map()

# vis_params = {
#     "bands": [ET_BAND_NAME],
#     "min": 0,
#     "max": 3,
#     "palette": "viridis",
# }

# Map.addLayer(image, vis_params)
# Map.addLayer(aoi, {"color": "red"}, "AOI")
# Map.centerObject(aoi, 13)
# Map

In [21]:
jurisdictions = ee.FeatureCollection(PATH_TO_JURISDICTION)

In [22]:
landuse_collection = ee.FeatureCollection(PATH_TO_LANDUSE)

# Check if the collection has "nutzung" property, otherwise rename it
# properties = landuse_collection.first().propertyNames().getInfo()

# print(f"Properties of the landuse collection: {properties}")

landuse_collection = (
    landuse_collection.map(lambda f: f.set("nutzung", f.get(LANDUSE_PROPERTY_NAME)))
    if LANDUSE_PROPERTY_NAME != "nutzung"
    else landuse_collection
)

print(
    f"Renamed {LANDUSE_PROPERTY_NAME} to 'nutzung'"
    if LANDUSE_PROPERTY_NAME != "nutzung"
    else "Collection has 'nutzung' property"
)

Collection has 'nutzung' property


## 2. Compute ET_green

In [51]:
process_et_green(
    et_collection_list=et_collection_list,
    landuse_collection=landuse_collection,
    jurisdictions=jurisdictions,
    double_cropping_image=double_cropping_image,
    year=YEAR,
    aoi=aoi,
    asset_path=ET_GREEN_ASSET_PATH,
    et_band_name=ET_BAND_NAME,
    time_step_type=TEMPORAL_RESOLUTION,
    resolution=ET_BAND_RESOLUTION,
    minimum_field_size=MINIMUM_FIELD_SIZE,
)

Exporting ET_green_dekadal_2018_04_D1 for 2018 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_dekadal_from_WaPOR_10m_v2/ET_green_dekadal_2018_04_D1
Exporting ET_green_dekadal_2018_04_D2 for 2018 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_dekadal_from_WaPOR_10m_v2/ET_green_dekadal_2018_04_D2
Exporting ET_green_dekadal_2018_04_D3 for 2018 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_dekadal_from_WaPOR_10m_v2/ET_green_dekadal_2018_04_D3
Exporting ET_green_dekadal_2018_05_D1 for 2018 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_dekadal_from_WaPOR_10m_v2/ET_green_dekadal_2018_05_D1
Exporting ET_green_dekadal_2018_05_D2 for 2018 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_dekadal_from_WaPOR_10m_v2/ET_green_dekadal_2018_05_D2
Exporting ET_green_dekadal_2018_05_D3 for 2018 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_dekadal_from_WaP

In [ ]:
# process_et_green_std(
#     et_collection_list=et_collection_list,
#     landuse_collection=landuse_collection,
#     jurisdictions=jurisdictions,
#     double_cropping_image=double_cropping_image,
#     year=YEAR,
#     aoi=aoi,
#     asset_path=ET_GREEN_STD_ASSET_PATH,
#     et_band_name=ET_BAND_NAME,
#     time_step_type=TEMPORAL_RESOLUTION,
#     resolution=ET_BAND_RESOLUTION,
#     minimum_field_size=MINIMUM_FIELD_SIZE,
# )

Exporting ET_green_dekadal_2019_04_D1 for 2019 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_std_dekadal_from_WaPOR_10m_v2/ET_green_dekadal_2019_04_D1
Exporting ET_green_dekadal_2019_04_D2 for 2019 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_std_dekadal_from_WaPOR_10m_v2/ET_green_dekadal_2019_04_D2
Exporting ET_green_dekadal_2019_04_D3 for 2019 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_std_dekadal_from_WaPOR_10m_v2/ET_green_dekadal_2019_04_D3
Exporting ET_green_dekadal_2019_05_D1 for 2019 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_std_dekadal_from_WaPOR_10m_v2/ET_green_dekadal_2019_05_D1
Exporting ET_green_dekadal_2019_05_D2 for 2019 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_std_dekadal_from_WaPOR_10m_v2/ET_green_dekadal_2019_05_D2
Exporting ET_green_dekadal_2019_05_D3 for 2019 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_gr

In [17]:
assetdir = f"projects/thurgau-irrigation/assets/{'FribourgAndVaud/ET_blue_postprocessed/ET_blue_postprocessed_dekadal_from_Landsat_30m_v2'}"


assets = ee.data.listAssets({"parent": assetdir})["assets"]
for img in assets:
    id = img['id']
    asset = ee.data.getAsset(id)
    name = id.split('ET_blue_postprocessed_dekadal_from_Landsat_30m_v2/')[1]
    # if (name.find('2021') > 0):
    print('asset name: ' + name)
    ee.data.deleteAsset(id)

    # ee.data.copyAsset(id, 'projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_postprocessed/ET_blue_postprocessed_dekadal_from_Landsat_30m_v2/' + name)


asset name: ET_blue_postprocessed_dekadal_2019_04_D1
asset name: ET_blue_postprocessed_dekadal_2019_04_D2
asset name: ET_blue_postprocessed_dekadal_2019_04_D3
asset name: ET_blue_postprocessed_dekadal_2019_05_D1
asset name: ET_blue_postprocessed_dekadal_2019_05_D2
asset name: ET_blue_postprocessed_dekadal_2019_05_D3
asset name: ET_blue_postprocessed_dekadal_2019_06_D1
asset name: ET_blue_postprocessed_dekadal_2019_06_D2
asset name: ET_blue_postprocessed_dekadal_2019_06_D3
asset name: ET_blue_postprocessed_dekadal_2019_07_D1
asset name: ET_blue_postprocessed_dekadal_2019_07_D2
asset name: ET_blue_postprocessed_dekadal_2019_07_D3
asset name: ET_blue_postprocessed_dekadal_2019_08_D1
asset name: ET_blue_postprocessed_dekadal_2019_08_D2
asset name: ET_blue_postprocessed_dekadal_2019_08_D3
asset name: ET_blue_postprocessed_dekadal_2019_09_D1
asset name: ET_blue_postprocessed_dekadal_2019_09_D2
asset name: ET_blue_postprocessed_dekadal_2019_09_D3
asset name: ET_blue_postprocessed_dekadal_2019

In [50]:
et_green_list = []


not_irrigated_crops = get_crops_to_exclude()
rainfed_crops = get_rainfed_reference_crops()

# Prepare rainfed fields
rainfed_fields = prepare_rainfed_fields(
    landuse_collection, double_cropping_image, not_irrigated_crops, rainfed_crops, MINIMUM_FIELD_SIZE
)

for img in range(NUMBER_OF_IMAGES):
    et_img = ee.Image(et_collection_list.get(img)).select(ET_BAND_NAME)

    et_green_img = compute_et_green(
        et_img, rainfed_fields, jurisdictions, et_band_name=ET_BAND_NAME
    )
    et_green_list.append(et_green_img)

In [13]:
# rainfed_fields.first().getInfo()

# distinct_features = rainfed_fields.distinct("nutzung_normalized")

# # Get a list of just the 'nutzung' values
# nutzung_values = distinct_features.aggregate_array("nutzung_normalized").getInfo()

# nutzung_values

# landuse_collection.first().getInfo()

In [51]:
Map = geemap.Map()

image = ee.Image(et_green_list[14])
image_ET = ee.Image(et_collection_list.get(14))

print(image.date().format("YYYY-MM-dd").getInfo())

vis_params = {
    "bands": ["ET_green"],
    "min": 0,
    "max": 3,
    "palette": "viridis",
}

vis_params_ET = {
    "bands": [ET_BAND_NAME],
    "min": 0,
    "max": 3,
    "palette": "viridis",
}



Map.center_object(aoi, 12)
Map.addLayer(image, vis_params, "ET green 10m")
Map.add_colorbar(vis_params, label="ET green [mm/month]", layer_name="ET green 10m")
Map.addLayer(image_ET, vis_params_ET, "ET 10m")
Map.addLayer(rainfed_fields.filterBounds(jurisdictions), {"color": "red"}, "Rainfed fields")
Map.addLayer(jurisdictions, {"color": "black"}, "jurisdictions")




Map

2019-08-21


Map(center=[46.78675332740657, 6.874554339425382], controls=(WidgetControl(options=['position', 'transparent_b…

### Comparing WaPOR to Landsat

In [15]:
# wapor_collection = ee.ImageCollection(
#     "projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_2019_dekadal_from_WaPOR_10m"
# ).map(lambda img: back_to_float(img, 100))

# landsat_collection = ee.ImageCollection(
#     "projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_2019_dekadal_from_Landsat_30m"
# ).map(lambda img: back_to_float(img, 100))

In [16]:
# print_collection_dates(wapor_collection)
# print_collection_dates(landsat_collection)

In [17]:
# wapor_list = wapor_collection.toList(wapor_collection.size())
# landsat_list = landsat_collection.toList(landsat_collection.size())

In [18]:
# wapor_august = ee.Image(wapor_list.get(14))
# landsat_august = ee.Image(landsat_list.get(14))

# Map = geemap.Map()

# vis_params = {
#     "bands": ["ET_green"],
#     "min": 0,
#     "max": 3,
#     "palette": "viridis",
# }

# Map.center_object(aoi, 12)
# Map.addLayer(wapor_august, vis_params, "ET WaPOR")
# Map.addLayer(landsat_august, vis_params, "ET Landsat")
# Map.add_colorbar(vis_params, label="ET green [mm/month]")

# # Print the date of the images
# print(f"WaPOR image date: {ee.Image(wapor_august).date().format('YYYY-MM-dd').getInfo()}")

# Map

In [19]:
# landsat_august.date().format("YYYY-MM-dd").getInfo()